<a href="https://colab.research.google.com/github/cuberisu/2024project/blob/main/llama2_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chat with Multiple PDF's Llama2 + Pinecone API + LangChain


## Step 01: Install All the Required Packages
일단 이 프로젝트 내에서는 한 번만 설치하면 된다.  
런타임 > 세션 다시 시작 해도 이미 설치됨.  
충격적인 것은 colab 내에서  
CPU와 GPU가 각각 사용하는 패키지 버전이 다르다는 점이다!

In [ ]:
! pip install langchain
! pip install pinecone-client
! pip install sentence_transformers
! pip install pdf2image
! pip install pypdf
! pip install xformers
! pip install bitsandbytes accelerate transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.8/812.8 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.8/276.8 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 22.5 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.9/215.9 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 48.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# ! pip install accelerate
# ! pip install -i https://pypi.org/simple/ bitsandbytes

Looking in indexes: https://pypi.org/simple/


## pip version, upgrade, and torch test

In [ ]:
!pip --version

pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)


In [ ]:
!pip install --upgrade pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 16.1 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2


In [ ]:
!torch
''' Why?
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.2.1+cu121 requires torch==2.2.1, but you have torch 2.2.2 which is incompatible.
torchtext 0.17.1 requires torch==2.2.1, but you have torch 2.2.2 which is incompatible.
torchvision 0.17.1+cu121 requires torch==2.2.1, but you have torch 2.2.2 which is incompatible.
Successfully installed torch-2.2.2 xformers-0.0.25.post1
'''

/bin/bash: line 1: torch: command not found


## Step 02: Import All the Required Libraries

In [ ]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from pdf2image import convert_from_path
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline
import os
import sys

In [ ]:
from langchain import HuggingFacePipeline, PromptTemplate
from langchain.chains import RetrievalQA

## Step 03: Load the PDF Files

In [ ]:
!mkdir pdfs # 이것도 세션 다시 시작해도 여전히 남아 있다.

In [ ]:
!gdown 1hPQlXrX8FbaYaLypxTmeVOFNitbBMlEE -O pdfs/yolov7paper.pdf
!gdown 1vILwiv6nS2wI3chxNabMgry3qnV67TxM -O pdfs/rachelgreecv.pdf

Downloading...
From: https://drive.google.com/uc?id=1hPQlXrX8FbaYaLypxTmeVOFNitbBMlEE
To: /content/pdfs/yolov7paper.pdf
100% 2.27M/2.27M [00:00<00:00, 194MB/s]
Downloading...
From: https://drive.google.com/uc?id=1vILwiv6nS2wI3chxNabMgry3qnV67TxM
To: /content/pdfs/rachelgreecv.pdf
100% 271k/271k [00:00<00:00, 116MB/s]


## Step 04: Extract the text from the PDF's

In [ ]:
loader = PyPDFDirectoryLoader('pdfs')
data = loader.load()
data

[Document(page_content='YOLOv7: Trainable bag-of-freebies sets new state-of-the-art for real-time object\ndetectors\nChien-Yao Wang1, Alexey Bochkovskiy, and Hong-Yuan Mark Liao1\n1Institute of Information Science, Academia Sinica, Taiwan\nkinyiu@iis.sinica.edu.tw, alexeyab84@gmail.com, and liao@iis.sinica.edu.tw\nAbstract\nYOLOv7 surpasses all known object detectors in both\nspeed and accuracy in the range from 5 FPS to 160 FPS\nand has the highest accuracy 56.8% AP among all known\nreal-time object detectors with 30 FPS or higher on GPU\nV100. YOLOv7-E6 object detector (56 FPS V100, 55.9%\nAP) outperforms both transformer-based detector SWIN-\nL Cascade-Mask R-CNN (9.2 FPS A100, 53.9% AP) by\n509% in speed and 2% in accuracy, and convolutional-\nbased detector ConvNeXt-XL Cascade-Mask R-CNN (8.6\nFPS A100, 55.2% AP) by 551% in speed and 0.7% AP\nin accuracy, as well as YOLOv7 outperforms: YOLOR,\nYOLOX, Scaled-YOLOv4, YOLOv5, DETR, Deformable\nDETR, DINO-5scale-R50, ViT-Adapter-B and

## Step 05: Split the Extracted Data into Text Chunks

In [ ]:
test_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)

In [ ]:
docs = test_splitter.split_documents(data)

In [ ]:
print(len(docs))

168


In [ ]:
docs[16]

Document(page_content='parameterization to obtain the ﬁnal inference model. One\nis to train multiple identical models with different train-\ning data, and then average the weights of multiple trained\nmodels. The other is to perform a weighted average of the\nweights of models at different iteration number. Module-\nlevel re-parameterization is a more popular research issue\nrecently. This type of method splits a module into multi-\nple identical or different module branches during training', metadata={'source': 'pdfs/yolov7paper.pdf', 'page': 1})

## Step 06: Download the Embeddings From the Hugging Face

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
query_result = embeddings.embed_query("Hello World")
print("Length", len(query_result))

Length 384


## Step 07: Initializing the Pinecone -> init is no longer...

In [ ]:
# 세션이 다시 시작되면 다시 실행해야 하는 것 같다!
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY', '06f14569-b4f5-4445-a214-79f6d97a54f2')  # find at app.pinecone.io
PINECONE_API_ENV = os.environ.get('PINECONE_API_ENV', 'gcp-starter')  # next to api key in console

In [ ]:
# Old code (2023.09.07)
# initialize pinecone
pinecone.init(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_API_ENV
)
index_name = "pdfreader2"  # put in the name of your pinecone index here

In [ ]:
''' New code (2023.09.07 -> 2024.04.06)
AttributeError: init is no longer a top-level attribute of the pinecone package.
Please create an instance of the Pinecone class instead.
Example:
'''

import os
from pinecone import Pinecone, PodSpec

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = 'pdfreader2'
# api_key = PINECONE_API_KEY

# pc.create_index(
#     name='pdfreader2', # my index name
#     dimension=384,  # my dimension
#     metric='cosine',  # my metric
#     spec=PodSpec(
#         environment = 'gcp-starter',
#         pod_type='starter',
#         pods=1
#     )
# )

if 'pdfreader2' not in pc.list_indexes().names():
    pc.create_index(
        name='pdfreader2',
        dimension=384,
        metric='cosine',
        spec=PodSpec(
            environment = 'gcp-starter',
            pod_type='starter',
            pods=1
            )
    )

In [ ]:
# Delete an index
# from pinecone import Pinecone, PodSpec
# pc = Pinecone(api_key="06f14569-b4f5-4445-a214-79f6d97a54f2")
# pc.delete_index("pdfreader")  # Resource pdfreader not found...
pc.delete_index("pdfreader2")

In [ ]:
# If you are using Pinecone serverless and getting the error
# "AttributeError: module 'pinecone' has no attribute 'init',
# first check that you are using the latest version of the Python client.

# You can check the version of the client by running:
!pip list pinecone-client
# pinecone-client: v3.2.2
# langchain-pinecone: none

## Step 08: Create Embeddings for each of the Text Chunk
이거 전에 Upsert vectors를 해야하나?

In [ ]:
# Upsert vectors
# 이제 인덱스를 만들었으므로 샘플 벡터를 2개의 고유한 네임스페이스에 삽입하세요 .

# 네임스페이스를 사용하면 단일 인덱스 내에서 벡터를 분할할 수 있습니다.
# 선택 사항이지만 네임스페이스별로 필터링할 수 있는 쿼리 속도를 높이고
# 다중 테넌트 요구 사항을 준수하기 위한 모범 사례입니다.

# index = pc.Index("pdfreader2")

In [ ]:
!pip install -U langchain-pinecone

In [ ]:
import os
os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY
# PineconeConfigurationError: You haven't specified an Api-Key.

# AttributeError: type object 'Pinecone' has no attribute 'from_texts'
# > use vectorstore?
from langchain.vectorstores import Pinecone

docsearch = Pinecone.from_texts([t.page_content for t in docs], embeddings, index_name=index_name)

In [ ]:
# Pinecone has no attribute 'from_texts' errors occur when using an outdated version of LangChain.
# Previously, the Python classes for both LangChain and Pinecone had objects named Pinecone,
# but this is no longer an issue in the latest LangChain version.


## If you already have an index, you can load it like this

In [ ]:
docsearch = Pinecone.from_existing_index(index_name, embeddings)

AttributeError: type object 'Pinecone' has no attribute 'from_existing_index'

## Step 09: Similarity Search

In [ ]:
query = "YOLOv7 outperforms which models"

In [ ]:
docs = docsearch.similarity_search(query)
docs

[Document(page_content='From the results we see that if compared with YOLOv4,\nYOLOv7 has 75% less parameters, 36% less computation,\nand brings 1.5% higher AP. If compared with state-of-the-\nart YOLOR-CSP, YOLOv7 has 43% fewer parameters, 15%\nless computation, and 0.4% higher AP. In the performance\nof tiny model, compared with YOLOv4-tiny-31, YOLOv7-\ntiny reduces the number of parameters by 39% and the\namount of computation by 49%, but maintains the same AP.\nOn the cloud GPU model, our model can still have a higher'),
 Document(page_content='From the results we see that if compared with YOLOv4,\nYOLOv7 has 75% less parameters, 36% less computation,\nand brings 1.5% higher AP. If compared with state-of-the-\nart YOLOR-CSP, YOLOv7 has 43% fewer parameters, 15%\nless computation, and 0.4% higher AP. In the performance\nof tiny model, compared with YOLOv4-tiny-31, YOLOv7-\ntiny reduces the number of parameters by 39% and the\namount of computation by 49%, but maintains the same AP.\

## Step 10: Creating a Llama2 Model Wrapper

In [ ]:
from huggingface_hub import notebook_login
import torch

In [ ]:
notebook_login()

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf", use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:720: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
# ImportError: Using `bitsandbytes` 8-bit quantization requires Accelerate: `pip install accelerate`
# and the latest version of bitsandbytes: `pip install -i https://pypi.org/simple/ bitsandbytes`
# > But its for CPU running: change the environment to GPU. and the issue will go away anyway.
# > That is colab CPU and GPU uses different transformer version.
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             token=True,  # FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
                                             # Please use `token` instead.
                                             load_in_8bit=True)  # The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions.
                                             # Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [ ]:
pipe = pipeline("text-generation",
                model=model,
                tokenizer=tokenizer,
                torch_dtype=torch.bfloat16,
                device_map='auto',
                max_new_tokens=512,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id)

In [ ]:
llm = HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0.1})

## Step 11: Create a Prompt Template

In [ ]:
SYSTEM_PROMPT = """Use the following pieces of context to answer the question at the end,
If you don't know the answer, just say that you don't know, don't try to make up an answer."""

In [ ]:
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<<SYS>>\n\n"

In [ ]:
SYSTEM_PROMPT = B_SYS + SYSTEM_PROMPT + E_SYS

In [ ]:
instruction = """
{context}

Question: {question}
"""

In [ ]:
template = B_INST + SYSTEM_PROMPT + instruction + E_INST

In [ ]:
template

"[INST]<<SYS>>\nUse the following pieces of context to answer the question at the end, \nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n<<SYS>>\n\n\n{context}\n\nQuestion: {question}\n[/INST]"

In [ ]:
prompt = PromptTemplate(template=template, input_variables=["context", "question"])

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt})

In [ ]:
result = qa_chain("YOLOv7 is used for")
# LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [ ]:
result['result']

'[INST]<<SYS>>\nUse the following pieces of context to answer the question at the end, \nIf you don\'t know the answer, just say that you don\'t know, don\'t try to make up an answer.\n<<SYS>>\n\n\ntion, in terms of the amount of parameters and computation,\nYOLOv7-X reduces 22% of parameters and 8% of compu-\ntation compared to YOLOv5-X (r6.1), but improves AP by\n2.2%.\n7\n\ntion, in terms of the amount of parameters and computation,\nYOLOv7-X reduces 22% of parameters and 8% of compu-\ntation compared to YOLOv5-X (r6.1), but improves AP by\n2.2%.\n7\n\nQuestion: YOLOv7 is used for\n[/INST]  Based on the provided context, the answer to the question "YOLOv7 is used for" is:\n\nYOLOv7 is used for object detection.\n\nThe context explains that YOLOv7-X reduces the number of parameters and computation compared to YOLOv5-X, but improves the AP (average precision) by 2.2%. This suggests that YOLOv7-X is a more efficient and accurate version of YOLOv5-X, which is likely used for object dete

In [ ]:
while True:
  user_input=input(f"prompt:")
  if user_input=='exit':
    print('Exiting')
    sys.exit()
  if user_input=='':
    continue
  result=qa_chain({'query': user_input})
  print(f"Answer:{result['result']}")

prompt:YOLOv7 is used for
Answer:[INST]<<SYS>>
Use the following pieces of context to answer the question at the end, 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
<<SYS>>


tion, in terms of the amount of parameters and computation,
YOLOv7-X reduces 22% of parameters and 8% of compu-
tation compared to YOLOv5-X (r6.1), but improves AP by
2.2%.
7

tion, in terms of the amount of parameters and computation,
YOLOv7-X reduces 22% of parameters and 8% of compu-
tation compared to YOLOv5-X (r6.1), but improves AP by
2.2%.
7

Question: YOLOv7 is used for
[/INST]  Based on the provided context, the answer to the question "YOLOv7 is used for" is:

YOLOv7 is used for object detection.

The context mentions that YOLOv7-X reduces the amount of parameters and computation compared to YOLOv5-X (r6.1), but improves AP (average precision) by 2.2%. This suggests that YOLOv7-X is a more efficient and accurate version of YOLOv5-X, and is likely used for obje

SystemExit: 

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
